# Problem

Users can submit a list of attributes for which they want to find the closest matching images.  User requests are captured as nested json.  Each image also has a list of attributes that apply to it, also stored as json.


EXAMPLE:

{'sex': 'male',
 'age': 25,
 'skin': {'wrinkles': 1, 'scars': True},
 'eyes': 'blue',
 'hair': {'colour': 'brown', 'texture': 'wavy', 'length': 'short'},
 'emotion': 'happy',
 'ears': 'Vulcan',
 'nose': 'red'
 }


Comparing nested json objects is very slow.  Calculating Levenshtein distance of an input json object with up to 22 attributes to 1 million existing json objects takes a very long time.  We need to speed this up.

# Assumptions

We are dealing with 1 million images, each of which has a json file containing a series of attributes which describe the image.

Each attribute has a set of possible values.

Some attributes may contain nested values.

Some attributes are categorical (colour) others boolean (1 or True)

Not every image has every attribute.



# Idea

Flatten the nested json objects and turn them into higher dimension vectors.

# Problems to resolve

### * most attributes can be resolved and flattened as strings but age will be an integer
    * this is probably easily resolved by just having each age be its own column (i.e. age.24, age.25)
    * depends on how Luc has age encoded
    * age almost certainly doesn't need to be exact - what does an image of a 26 year old look like vs 25?
### * ~~need to figure out how to transform True/False values like {wrinkles: 1}~~
### * is there a definitive list of attributes?

# Key Functions

In [64]:
from collections.abc import MutableMapping
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
import scipy.sparse
import json
import pickle

### Flatten via generator

Flattening functions totally stolen: https://www.freecodecamp.org/news/how-to-flatten-a-dictionary-in-python-in-4-different-ways/

Using the generator option is much more memory efficient

#### sample\['hair'\]\['color'\] :'brown' becomes 'hair.color.brown'

In [20]:
def flatten_dict_gen(d, parent_key, sep):
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):      # testing if the value is itself a mutable key/value object
            yield from flatten_dict(v, new_key, sep=sep).items()
        else:
            yield new_key, v

In [21]:
def flatten_dict(d: MutableMapping, parent_key: str = '', sep: str = '.'):
    return dict(flatten_dict_gen(d, parent_key, sep))

### Convert flat json to string

Once we have a flat dictionary, we need to create combine the key/value pairs into a single string.  

Need to work around k/v pairs where the value is boolean.  The fact that the pair exists indicates that it was true in the old json.

In [22]:
def flat_to_string(in_dict):
    dict_string = " ".join([f"{k}.{v}" if v not in [0,1, True, False] else f"{k}" for k,v in in_dict.items()])
    return dict_string

# Load test file, flatten, convert to string, and vectorize

#### Load json file containing nested json objects

In [23]:
%%time
with open("sample_json_1000000.json", 'r') as fin:
    dict_list = json.load(fin)

CPU times: user 4.69 s, sys: 688 ms, total: 5.38 s
Wall time: 5.36 s


In [24]:
len(dict_list)

1000000

In [25]:
dict_list[0]

{'sex': 'female',
 'age': 74,
 'skin': {'wrinkles': True},
 'hair': {'colour': 'blonde', 'length': 'short', 'texture': 'straight'},
 'emotion': 'angry',
 'ears': 'big',
 'eyebrows': 'arched',
 'accessories': 'earrings'}

#### Flatten json objects and convert to list of strings

In [26]:
%%time
string_list = []
for item in dict_list:
    flat = flatten_dict(item)
    dict_string = flat_to_string(flat)
    #mystring = " ".join([f"{k}.{v}" if v not in [0,1, True, False] else f"{k}" for k,v in flat.items()])
    string_list.append(dict_string)

CPU times: user 8.37 s, sys: 86.8 ms, total: 8.45 s
Wall time: 8.45 s


In [27]:
len(string_list)

1000000

In [28]:
string_list[0]

'sex.female age.74 skin.wrinkles hair.colour.blonde hair.length.short hair.texture.straight emotion.angry ears.big eyebrows.arched accessories.earrings'

#### Vectorize the list of strings

In [29]:
vectorizer = CountVectorizer(token_pattern='\S+', binary=True)

In [30]:
%%time
X = vectorizer.fit_transform(string_list)

CPU times: user 6.14 s, sys: 200 ms, total: 6.34 s
Wall time: 6.33 s


In [31]:
X

<1000000x102 sparse matrix of type '<class 'numpy.int64'>'
	with 6205181 stored elements in Compressed Sparse Row format>

In [32]:
Y = vectorizer.get_feature_names()

In [33]:
len(Y)

102

#### Complete list of attributes

In [34]:
Y

['accessories.earrings',
 'accessories.glasses',
 'accessories.hat',
 'age.10',
 'age.11',
 'age.12',
 'age.13',
 'age.14',
 'age.15',
 'age.16',
 'age.17',
 'age.18',
 'age.19',
 'age.20',
 'age.21',
 'age.22',
 'age.23',
 'age.24',
 'age.25',
 'age.26',
 'age.27',
 'age.28',
 'age.29',
 'age.30',
 'age.31',
 'age.32',
 'age.33',
 'age.34',
 'age.35',
 'age.36',
 'age.37',
 'age.38',
 'age.39',
 'age.40',
 'age.41',
 'age.42',
 'age.43',
 'age.44',
 'age.45',
 'age.46',
 'age.47',
 'age.48',
 'age.49',
 'age.50',
 'age.51',
 'age.52',
 'age.53',
 'age.54',
 'age.55',
 'age.56',
 'age.57',
 'age.58',
 'age.59',
 'age.60',
 'age.61',
 'age.62',
 'age.63',
 'age.64',
 'age.65',
 'age.66',
 'age.67',
 'age.68',
 'age.69',
 'age.70',
 'age.71',
 'age.72',
 'age.73',
 'age.74',
 'age.75',
 'age.76',
 'age.77',
 'age.78',
 'age.79',
 'age.80',
 'ears.big',
 'ears.droopy',
 'ears.huge',
 'emotion.angry',
 'emotion.happy',
 'emotion.sad',
 'ethnicity.asian',
 'ethnicity.black',
 'ethnicity.cau

#### Save the list of attributes to file

In [ ]:
with open("attributes.pickle", "wb") as fout:
    pickle.dump(Y,fout)

### Save the vectorizer for future use

In [68]:
with open("vectorizer.pickle", "wb") as fout:
    pickle.dump(vectorizer,fout)

In [70]:
with open("vectorizer.pickle", "rb") as fin3:
    X3 =pickle.load(fin3)

# Find similar vector in array

### TODO:
#### This is too expensive.  Instead use the numpy sparse matrix

### Get array from vectorized results

In [35]:
x_array = X.toarray()

In [36]:
x_array.shape

(1000000, 102)

In [37]:
x_array

array([[1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0]])

### Extract row as test vector

In [38]:
test_index = 375

In [39]:
test_row = x_array[test_index : test_index+1,:]

In [40]:
test_row

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

## Find the nearest matches in the matrix for the test vector

In [41]:
from scipy.spatial import distance

In [42]:
%%time
distances = distance.cdist(test_row, x_array, "cosine")[0]
five_closest = np.argsort(distances)[:5]  # get N closest matches
#closest_match = np.argmin(distances) # this gives index of closest match



CPU times: user 512 ms, sys: 405 ms, total: 917 ms
Wall time: 910 ms


In [43]:
five_closest

array([   375, 432047, 553410,  43855, 712138])

In [44]:
distances[375]

2.220446049250313e-16

### You can save the entire 1M record matrix as a numpy array

In [ ]:
x_array.shape

In [ ]:
with open("image_matrix", 'wb') as fout2:
    np.save(fout2, x_array)

In [ ]:
with open("image_matrix", 'rb') as fin3:
    new_x_array = np.load(fin3)

In [ ]:
new_x_array.shape

## Take an input json file and check it against the 1M record matrix

Note: this seems hacky.  There may be a better way to compare the vector of the input json (which will only contain a few columns) against the 102 column rows of the matrix.  This sounds like a question for Sir HEALY, Earl of Embedding.

In [45]:
from collections import defaultdict

#### Load list of all attributes from file

In [47]:
with open("attributes.pickle", "rb") as fin2:
    attributes = pickle.load(fin2)

#### Create dict with all attributes as keys with 0 values

In [48]:
attributes_dict = defaultdict.fromkeys(attributes, 0)

#### Get user input as json

In [49]:
input_json = {
    'sex' : 'male',
    'age' : 55,
    'ears' : 'big',
    'hair' : {'colour':'blonde' }
}

#### Flatten user input and convert to string

In [50]:
flat_input = flatten_dict(input_json)

In [51]:
input_str = flat_to_string(flat_input)

In [52]:
input_str

'sex.male age.55 ears.big hair.colour.blonde'

#### Iterate over attributes in the string and change  respective values in attributes_dict to 1

In [53]:
for attribute in input_str.split():
    attributes_dict[attribute] = 1

In [54]:
attributes_dict

defaultdict(None,
            {'accessories.earrings': 0,
             'accessories.glasses': 0,
             'accessories.hat': 0,
             'age.10': 0,
             'age.11': 0,
             'age.12': 0,
             'age.13': 0,
             'age.14': 0,
             'age.15': 0,
             'age.16': 0,
             'age.17': 0,
             'age.18': 0,
             'age.19': 0,
             'age.20': 0,
             'age.21': 0,
             'age.22': 0,
             'age.23': 0,
             'age.24': 0,
             'age.25': 0,
             'age.26': 0,
             'age.27': 0,
             'age.28': 0,
             'age.29': 0,
             'age.30': 0,
             'age.31': 0,
             'age.32': 0,
             'age.33': 0,
             'age.34': 0,
             'age.35': 0,
             'age.36': 0,
             'age.37': 0,
             'age.38': 0,
             'age.39': 0,
             'age.40': 0,
             'age.41': 0,
             'age.42': 0,
          

#### Create a 2D array of the values in the attributes_dict

As mentioned, this is a real hack

In [55]:
input_vector = np.array([list(attributes_dict.values())])

In [56]:
input_vector[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [62]:
%%time
input_distances = distance.cdist(input_vector, x_array, "cosine")[0]
five_closest = np.argsort(input_distances)[:5]  # get N closest matches

CPU times: user 514 ms, sys: 404 ms, total: 918 ms
Wall time: 917 ms


In [58]:
five_closest

array([849770, 356286, 882888, 572881, 753306])

In [60]:
input_distances[849770]

0.10557280900008414

In [61]:
x_array[771755]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0])

#### Save the sparse matrix
This only has to be redone if the image database and the descriptive json objects change

In [65]:
scipy.sparse.save_npz('database_matrix.npz', X)

In [66]:
# you can reload the matrix with:
# X = scipy.sparse.load_npz('database_matrix.npz')